E7. Transportation Simplex

1.Read an external file containing the parameters of a transportation problem LP problem
2.Different methods can be used to find the initial feasible solution for the problem:
Northwest corner rule
Minimum cost method
Minimum Row cost method
Vogel's method

In [1]:
import numpy as np
import pandas as pd

# Function to read transportation problem parameters from a TXT file
def read_transportation_params(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Extract supply, demand, and cost matrix
    supply_line = next(line for line in lines if 'Supply' in line)
    demand_line = next(line for line in lines if 'Demand' in line)
    cost_start_index = lines.index("Cost\n")  # Assuming exact 'Cost' line
    cost_lines = lines[cost_start_index + 1:]

    # Parse supply, demand, and cost matrix
    supply = np.array([int(val.strip()) for val in supply_line.split(",")[1:]])
    demand = np.array([int(val.strip()) for val in demand_line.split(",")[1:]])

    cost_matrix = []
    for cost_line in cost_lines:
        if cost_line.strip():  
            cost_values = [float(val.strip()) for val in cost_line.split(",")]
            cost_matrix.append(cost_values)

    cost = np.array(cost_matrix)

    return supply, demand, cost


# Function to implement the Northwest Corner Rule
def northwest_corner_rule(supply, demand, cost):
    n = len(supply)
    m = len(demand)
    x = np.zeros((n, m))
    i, j = 0, 0

    while i < n and j < m:
        allocation = min(supply[i], demand[j])
        x[i, j] = allocation
        supply[i] -= allocation
        demand[j] -= allocation
        if supply[i] == 0:
            i += 1
        elif demand[j] == 0:
            j += 1
    
    total_cost = np.sum(x * cost)
    return x, total_cost


# Function to implement the Minimum Cost Method 
def minimum_cost_method(supply, demand, cost):
    n = len(supply)
    m = len(demand)
    x = np.zeros((n, m))
    remaining_supply = supply.copy()
    remaining_demand = demand.copy()

    cost_copy = cost.copy()  
    while np.sum(remaining_supply) > 0:
        min_cost = np.min(cost_copy)
        indices = np.where(cost_copy == min_cost)
        i, j = indices[0][0], indices[1][0]
        
        allocation = min(remaining_supply[i], remaining_demand[j])
        x[i, j] = allocation
        remaining_supply[i] -= allocation
        remaining_demand[j] -= allocation
        
        cost_copy[i, j] = float('inf')  
    
    total_cost = np.sum(x * cost)
    return x, total_cost


# Function to implement the Minimum Row Cost Method
def minimum_row_cost_method(supply, demand, cost):
    n = len(supply)
    m = len(demand)
    x = np.zeros((n, m))
    remaining_supply = supply.copy()
    remaining_demand = demand.copy()

    cost_copy = cost.copy()
    for i in range(n):
        while remaining_supply[i] > 0:
            min_cost = np.min(cost_copy[i])
            j = np.where(cost_copy[i] == min_cost)[0][0]
            allocation = min(remaining_supply[i], remaining_demand[j])
            x[i, j] = allocation
            remaining_supply[i] -= allocation
            remaining_demand[j] -= allocation
            
            # Mark this cell as used to avoid re-selection
            cost_copy[i, j] = float('inf')
    
    total_cost = np.sum(x * cost)
    return x, total_cost


# Function to implement Vogel's Approximation Method
def vogels_approximation_method(supply, demand, cost):
    n = len(supply)
    m = len(demand)
    x = np.zeros((n, m))
    remaining_supply = supply.copy()
    remaining_demand = demand.copy()

    cost_copy = cost.copy()
    while np.sum(remaining_supply) > 0:
        row_penalty = []
        col_penalty = []

        for i in range(n):
            available_costs = cost_copy[i][cost_copy[i] != float('inf')]
            if len(available_costs) > 1:
                row_penalty.append(np.sort(available_costs)[1] - np.sort(available_costs)[0])
            else:
                row_penalty.append(0)

        for j in range(m):
            available_costs = cost_copy[:, j][cost_copy[:, j] != float('inf')]
            if len(available_costs) > 1:
                col_penalty.append(np.sort(available_costs)[1] - np.sort(available_costs)[0])
            else:
                col_penalty.append(0)

        if max(row_penalty) >= max(col_penalty):
            i = np.argmax(row_penalty)  
            min_cost = np.min(cost_copy[i])
            j = np.where(cost_copy[i] == min_cost)[0][0]
        else:
            j = np.argmax(col_penalty)  
            min_cost = np.min(cost_copy[:, j])
            i = np.where(cost_copy[:, j] == min_cost)[0][0]

        allocation = min(remaining_supply[i], remaining_demand[j])
        x[i, j] = allocation
        remaining_supply[i] -= allocation
        remaining_demand[j] -= allocation
        
        cost_copy[i, j] = float('inf')

    total_cost = np.sum(x * cost)
    return x, total_cost


file_path = '/Users/mariusleorat/Downloads/E7file2.txt'  
supply, demand, cost = read_transportation_params(file_path)

northwest_solution, northwest_cost = northwest_corner_rule(supply.copy(), demand.copy(), cost.copy())
min_cost_solution, min_cost_total = minimum_cost_method(supply.copy(), demand.copy(), cost.copy())
min_row_cost_solution, min_row_cost_total = minimum_row_cost_method(supply.copy(), demand.copy(), cost.copy())
vogels_solution, vogels_cost = vogels_approximation_method(supply.copy(), demand.copy(), cost.copy())

# Display results for each method
print("Northwest Corner Rule:")
print("Solution Matrix:")
print(northwest_solution)
print("Total Cost:", northwest_cost)

print("\nMinimum Cost Method:")
print("Solution Matrix:")
print(min_cost_solution)
print("Total Cost:", min_cost_total)

print("\nMinimum Row Cost Method:")
print("Solution Matrix:")
print(min_row_cost_solution)
print("Total Cost:", min_row_cost_total)

print("\nVogel's Approximation Method:")
print("Solution Matrix:")
print(vogels_solution)
print("Total Cost:", vogels_cost)


Northwest Corner Rule:
Solution Matrix:
[[20.  0.  0.]
 [ 5. 25.  0.]
 [ 0.  0. 25.]]
Total Cost: 905.0

Minimum Cost Method:
Solution Matrix:
[[ 0. 20.  0.]
 [ 5.  0. 25.]
 [20.  5.  0.]]
Total Cost: 665.0

Minimum Row Cost Method:
Solution Matrix:
[[ 0. 20.  0.]
 [ 5.  0. 25.]
 [20.  5.  0.]]
Total Cost: 665.0

Vogel's Approximation Method:
Solution Matrix:
[[20.  0.  0.]
 [ 5.  0. 25.]
 [ 0. 25.  0.]]
Total Cost: 605.0


3. The transportation simplex algorithm in an arbitrary transportation problem.

In [3]:
import numpy as np
import pandas as pd
import itertools

# Function to read transportation problem parameters from a TXT file
def read_transportation_params(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Extract supply, demand, and cost matrix
    supply_line = next(line for line in lines if 'Supply' in line)
    demand_line = next(line for line in lines if 'Demand' in line)
    cost_start_index = lines.index("Cost\n")
    cost_lines = lines[cost_start_index + 1:]

    # Parse supply, demand, and cost matrix
    supply = np.array([int(val.strip()) for val in supply_line.split(",")[1:]])
    demand = np.array([int(val.strip()) for val in demand_line.split(",")[1:]])

    cost_matrix = []
    for cost_line in cost_lines:
        if cost_line.strip():  
            cost_values = [float(val.strip()) for val in cost_line.split(",")]
            cost_matrix.append(cost_values)

    cost = np.array(cost_matrix)

    return supply, demand, cost


# Function to find an initial feasible solution using the Northwest Corner Rule
def northwest_corner_rule(supply, demand, cost):
    n = len(supply)
    m = len(demand)
    x = np.zeros((n, m))
    i, j = 0, 0

    while i < n and j < m:
        allocation = min(supply[i], demand[j])
        x[i, j] = allocation
        supply[i] -= allocation
        demand[j] -= allocation
        if supply[i] == 0:
            i += 1
        elif demand[j] == 0:
            j += 1
    
    return x

def calculate_potentials(x, cost):
    n = x.shape[0]
    m = x.shape[1]

    u = [float('inf')] * n
    v = [float('inf')] * m

    u[0] = 0

    while True:
        updated = False  
        for i in range(n):
            for j in range(m):
                if x[i, j] > 0: 
                    if u[i] != float('inf') and v[j] == float('inf'):
                        v[j] = cost[i, j] - u[i]
                        updated = True
                    elif v[j] != float('inf') and u[i] == float('inf'):
                        u[i] = cost[i, j] - v[j]
                        updated = True

        if not updated:  
            break

    
    reduced_costs = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            if x[i, j] == 0:  
                reduced_costs[i, j] = cost[i, j] - u[i] - v[j]

    return reduced_costs



def find_cycle(x, start):
   
    n = x.shape[0]
    m = x.shape[1]

    cycle = [start]
    visited = set()
    visited.add(start)

    while True:
        last = cycle[-1]
        found = False

        
        for j in range(m):
            if x[last[0], j] > 0 and (last[0], j) not in visited:
                cycle.append((last[0], j))
                visited.add((last[0], j))
                found = True
                break

        if not found:
            
            for i in range(n):
                if x[i, last[1]] > 0 and (i, last[1]) not in visited:
                    cycle.append((i, last[1]))
                    visited.add((i, last[1]))
                    found = True
                    break

        if not found or cycle[0] == cycle[-1]:
            break

    print("Cycle:", cycle)

    return cycle


def improve_solution(x, cycle):
    min_allocation = min(x[point] for point in cycle[1:-1:2])

    for i in range(len(cycle) - 1):
        if i % 2 == 0:
            x[cycle[i]] += min_allocation
        else:
            x[cycle[i]] -= min_allocation

    return x


def transportation_simplex(x, cost, max_iterations=5000):
    iteration = 0
    while iteration < max_iterations:
        iteration += 1
        print("Iteration:", iteration)
        
        reduced_costs = calculate_potentials(x, cost)

        pivot = np.unravel_index(np.argmin(reduced_costs), reduced_costs.shape)
        print("Pivot:", pivot)

        cycle = find_cycle(x, pivot)
        print("Cycle:", cycle)

        x = improve_solution(x, cycle)
        print("Improved solution:")
        print(x)

        if np.array_equal(x, improve_solution(x, cycle)):
            print("Optimal solution found.")
            break

    total_cost = np.sum(x * cost)

    return x, total_cost



# Read transportation parameters from a TXT file
file_path = '/Users/mariusleorat/Downloads/E7_file.txt'  # Replace with your file path
supply, demand, cost = read_transportation_params(file_path)

# Find an initial feasible solution using the Northwest Corner Rule
initial_solution = northwest_corner_rule(supply.copy(), demand.copy(), cost.copy())

# Apply the transportation simplex algorithm to find the optimal solution
optimal_solution, optimal_cost = transportation_simplex(initial_solution, cost)

# Display the results
print("Optimal Solution:")
print(optimal_solution)
print("Optimal Cost:", optimal_cost)

Iteration: 1
Pivot: (0, 2)
Cycle: [(0, 2), (0, 0), (1, 0), (1, 1)]
Cycle: [(0, 2), (0, 0), (1, 0), (1, 1)]
Improved solution:
[[ 0.  0. 20.]
 [25. 25.  0.]
 [ 0.  0. 25.]]
Optimal solution found.
Optimal Solution:
[[ 0.  0. 20.]
 [25. 25.  0.]
 [ 0.  0. 25.]]
Optimal Cost: 1125.0
